# Here is an example of how to use our model

***Please note that this is the best model we could developp ourselves based on the data driven approach. The task driven approach using ResNet50 used in Week5.ipynb gave better results however.***

***The best mmodel's weights file (.pth file) is around 400MB and can be downloaded from https://we.tl/t-uhOyMznu5R 
It has to be put in the same folder as this script***

First, loading the data:

In [3]:
import sys
sys.path.append('./../')
!{sys.executable} -m pip install gdown h5py

In [4]:
from week5.utils import load_it_data, visualize_img
import matplotlib.pyplot as plt
import numpy as np
import gdown
url = "https://drive.google.com/file/d/1s6caFNRpyR9m7ZM6XEv_e8mcXT3_PnHS/view?usp=share_link"
output = "IT_data.h5"
gdown.download(url, output, quiet=False, fuzzy=True)


Downloading...
From (uriginal): https://drive.google.com/uc?id=1s6caFNRpyR9m7ZM6XEv_e8mcXT3_PnHS
From (redirected): https://drive.google.com/uc?id=1s6caFNRpyR9m7ZM6XEv_e8mcXT3_PnHS&confirm=t&uuid=22ec00e7-51cc-4ae5-b279-d37dba243542
To: /home/jupyter/NX414-Project/week5/IT_data.h5
100%|██████████| 384M/384M [00:01<00:00, 225MB/s]  


'IT_data.h5'

In [5]:
path_to_data = '' ## Insert the folder where the data is, if you download in the same folder as this notebook then leave it blank

stimulus_train, stimulus_val, stimulus_test, objects_train, objects_val, objects_test, spikes_train, spikes_val = load_it_data(path_to_data)

In [6]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn

device = ("cuda" if torch.cuda.is_available() else "cpu")

**The class corresponding to our best model architecture:**

In [7]:
# import the necessary packages
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch import flatten

from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset
from torchvision.transforms import ToTensor
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import argparse
import torch
import time

class LeNet4Fc(Module):
	def __init__(self, numChannels, neurons):
		# call the parent constructor
		super(LeNet4Fc, self).__init__()
		# initialize first set of CONV => RELU => POOL layers
		self.conv1 = Conv2d(in_channels=numChannels, out_channels=20,
			kernel_size=(5, 5))
		self.relu1 = ReLU()
		self.maxpool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
		# initialize second set of CONV => RELU => POOL layers
		self.conv2 = Conv2d(in_channels=20, out_channels=50,
			kernel_size=(5, 5))
		self.relu2 = ReLU()
		self.maxpool2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
		# 4 sets of FC => RELU layers
		self.fc1 = Linear(in_features=140450, out_features=1000)
		self.relu3 = ReLU()
		self.fc2 = Linear(in_features=1000, out_features=1000)
		self.relu4 = ReLU()
		self.fc3 = Linear(in_features=1000, out_features=500)
		self.relu5 = ReLU()
		self.fc4 = Linear(in_features=500, out_features=neurons)
        
	def forward(self, x):
		# pass the input through our first set of CONV => RELU =>
		# POOL layers
		x = self.conv1(x)
		x = self.relu1(x)
		x = self.maxpool1(x)
		# pass the output from the previous layer through the second
		# set of CONV => RELU => POOL layers
		x = self.conv2(x)
		x = self.relu2(x)
		x = self.maxpool2(x)
		# flatten the output from the previous layer and pass it
		# through our only set of FC => RELU layers
		x = flatten(x, 1)
		x = self.fc1(x)
		x = self.relu3(x)
		#We add our two new fc layers
		x = self.fc2(x)
		x = self.relu4(x)
		x = self.fc3(x)
		x = self.relu5(x)        
		# output after the second fc layer
		output=self.fc4(x)
        
		# return the output predictions
		return output

**Now we load the weights:**

In [8]:
model = LeNet4Fc(numChannels=3,neurons=168)
model.load_state_dict(torch.load("best_model.zip"))

RuntimeError: [enforce fail at inline_container.cc:129] . file in archive is not in a subdirectory: best_model.pth

Converting validation set and test sets to tensors and sending everything to the right device:

In [7]:
stimulus_val = torch.from_numpy(stimulus_val).type(torch.FloatTensor)
spikes_val=torch.from_numpy(spikes_val).type(torch.FloatTensor)

stimulus_test=torch.from_numpy(stimulus_test).type(torch.FloatTensor)

stimulus_val=stimulus_val.to(device)
stimulus_test=stimulus_test.to(device)
model.to(device)

LeNet4Fc(
  (conv1): Conv2d(3, 20, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=140450, out_features=1000, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=1000, out_features=1000, bias=True)
  (relu4): ReLU()
  (fc3): Linear(in_features=1000, out_features=500, bias=True)
  (relu5): ReLU()
  (fc4): Linear(in_features=500, out_features=168, bias=True)
)

**For example, let's use our model to predict the spikes for the validation set:**

In [8]:
with torch.no_grad():
    pred_val=model(stimulus_val)


**We now have all the 288x168 predictions for the validation set in the tensor named pred_val.**